<a href="https://colab.research.google.com/github/dakotahansen11/IS_4487/blob/main/week11_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 4487 Module 5 Script

## Objectives

a. Understand model accuracy.  Why is it a performance metric for classification and not regression?
    
b. Calculate accuracy for a simple majority class model (this is the same as calculating the proportion of the majority class in a binary variable). Consider: x <- c(1, 1, 1, 0, 0).  What is the majority class? What is the proportion of the majority class in x?
    
c. Fit a tree model of the target with just one predictor variable and calculate the accuracy of this model.
    
d. Interpret a tree model, and calculate information gain.
    
e. Fit a tree model of the target using all the predictors, then:  create a visualization of the tree and identify the top 3 most important predictors in this model.
    
f. How do these models compare to majority class prediction?
    
g. How will you use a classification model as part of a solution to the AdviseInvest case?

We will use the MegaTelCo data for this demonstration.

##Load Libraries

In this class we will be using 
- Pandas
- Scikitlearn
- Matplotlib


In [1]:
import pandas as pd
import matplotlib as mpl
import numpy as np

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree


## Getting data into Pandas

In this case we will load data from the statsmodels.org library

See the Canvas assignments and lectures for a description of the Megatelco data

**Note:** you will need to enter a code supplied by Google in the next step. 


In [3]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

df = pd.read_csv (r'/content/gdrive/MyDrive/Colab Notebooks/daily_aqi_by_county_2021.csv')

Mounted at /content/gdrive


In [4]:
#look at the top rows
df.head(10) 

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2021-01-01,27,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2021-01-04,47,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2021-01-07,24,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2021-01-10,39,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2021-01-13,46,Good,PM2.5,01-003-0010,1
5,Alabama,Baldwin,1,3,2021-01-16,21,Good,PM2.5,01-003-0010,1
6,Alabama,Baldwin,1,3,2021-01-19,52,Moderate,PM2.5,01-003-0010,1
7,Alabama,Baldwin,1,3,2021-01-22,11,Good,PM2.5,01-003-0010,1
8,Alabama,Baldwin,1,3,2021-01-25,39,Good,PM2.5,01-003-0010,1
9,Alabama,Baldwin,1,3,2021-01-28,22,Good,PM2.5,01-003-0010,1


In [5]:
#look at the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218196 entries, 0 to 218195
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   State Name                 218196 non-null  object
 1   county Name                218196 non-null  object
 2   State Code                 218196 non-null  int64 
 3   County Code                218196 non-null  int64 
 4   Date                       218196 non-null  object
 5   AQI                        218196 non-null  int64 
 6   Category                   218196 non-null  object
 7   Defining Parameter         218196 non-null  object
 8   Defining Site              218196 non-null  object
 9   Number of Sites Reporting  218196 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 16.6+ MB


In [6]:
#describe the data before cleaning it
df.describe()

,State Code,County Code,AQI,Number of Sites Reporting
count,218196.000000,218196.000000,218196.000000,218196.000000
mean,30.178390,82.098265,39.584250,1.924220
std,15.755914,89.555906,21.547055,2.251967
min,1.000000,1.000000,0.000000,1.000000
25%,18.000000,25.000000,29.000000,1.000000
50%,30.000000,61.000000,38.000000,1.000000
75%,42.000000,111.000000,47.000000,2.000000
max,80.000000,810.000000,2723.000000,34.000000


## Clean up the data
Did you notice anything unusual about the "house" amounts? 

How about the handset price and income?

Clean up the data in a  new datafram named "df_clean"


In [9]:
#delete any rows with missing values in the clean dataframe
df_clean = df
df_clean = df_clean.dropna() 

df_clean.describe()

,State Code,County Code,AQI,Number of Sites Reporting
count,218196.000000,218196.000000,218196.000000,218196.000000
mean,30.178390,82.098265,39.584250,1.924220
std,15.755914,89.555906,21.547055,2.251967
min,1.000000,1.000000,0.000000,1.000000
25%,18.000000,25.000000,29.000000,1.000000
50%,30.000000,61.000000,38.000000,1.000000
75%,42.000000,111.000000,47.000000,2.000000
max,80.000000,810.000000,2723.000000,34.000000


# Standardize attributes

- Convert the "college" and "considering_change_of_plan" attributes to 0 or 1
- Convert non-numeric attributes to numeric scale of 1-3


In [10]:
#Get distinct values
df_clean['Defining Parameter'].unique() 

array(['PM2.5', 'Ozone', 'PM10', 'NO2', 'CO', 'SO2'], dtype=object)

In [14]:
df_clean.loc[df_clean['Defining Parameter'] == 'PM2.5', 'Defining Parameter'] = "1" 
df_clean.loc[df_clean['Defining Parameter'] == 'Ozone', 'Defining Parameter'] = "0"

df_clean['Defining Parameter'] = df_clean['Defining Parameter'].astype('string') 

df_clean.head(10) 

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2021-01-01,27,Good,1,01-003-0010,1
1,Alabama,Baldwin,1,3,2021-01-04,47,Good,1,01-003-0010,1
2,Alabama,Baldwin,1,3,2021-01-07,24,Good,1,01-003-0010,1
3,Alabama,Baldwin,1,3,2021-01-10,39,Good,1,01-003-0010,1
4,Alabama,Baldwin,1,3,2021-01-13,46,Good,1,01-003-0010,1
5,Alabama,Baldwin,1,3,2021-01-16,21,Good,1,01-003-0010,1
6,Alabama,Baldwin,1,3,2021-01-19,52,Moderate,1,01-003-0010,1
7,Alabama,Baldwin,1,3,2021-01-22,11,Good,1,01-003-0010,1
8,Alabama,Baldwin,1,3,2021-01-25,39,Good,1,01-003-0010,1
9,Alabama,Baldwin,1,3,2021-01-28,22,Good,1,01-003-0010,1


# Convert attributes to factors

- Leave
- College
- Reported satisfaction
- Reported usage level
- Considering change of plan

In [15]:
#Method #1
#df_clean['leave'] = pd.Categorical(df_clean['leave'])

#Method #2
df_clean['Category'] = df_clean['Category'].astype('category')
df_clean['Defining Parameter'] = df_clean['Defining Parameter'].astype('category')

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218196 entries, 0 to 218195
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   State Name                 218196 non-null  object  
 1   county Name                218196 non-null  object  
 2   State Code                 218196 non-null  int64   
 3   County Code                218196 non-null  int64   
 4   Date                       218196 non-null  object  
 5   AQI                        218196 non-null  int64   
 6   Category                   218196 non-null  category
 7   Defining Parameter         218196 non-null  category
 8   Defining Site              218196 non-null  object  
 9   Number of Sites Reporting  218196 non-null  int64   
dtypes: category(2), int64(4), object(4)
memory usage: 13.7+ MB


## Calculate distribution of the outcome  
What is the proportion of people who churned?

Why should we care about this proportion?

1. An important step in EDA is to understand the distribution of the target variable.

2. The majority class in the target variable will serve as an important benchmark for model performance. If we used what we'll call a "majority class classifier"---this consists in always predicting the majority class, which in this case is `STAY`---we would be correct 1 - .49 or 51% of the time.  Another way of saying this is that majority class classifier in the MegaTelCo case would result in accuracy of .51.  

Accuracy is defined as the proportion of correctly predicted labels. It is a commonly used error metric for evaluating classifier performance.

Think about why a majority class model in this case would have an accuracy of .51.

Whatever later model we develop should have better accuracy than this performance benchmark.

In [18]:
#Add new field with binary value for leave  
df_clean['category_flag'] = df_clean['Category'].str.replace('Hazardous','0')  
df_clean['category_flag'] = df_clean['category_flag'].str.replace('Very Unhealthy','1')
df_clean['category_flag'] = df_clean['category_flag'].str.replace('Unhealthy','2')
df_clean['category_flag'] = df_clean['category_flag'].str.replace('Unhealthy for sensitive groups','3')
df_clean['category_flag'] = df_clean['category_flag'].str.replace('Moderate','4')
df_clean['category_flag'] = df_clean['category_flag'].str.replace('Good','5')

#Convert to integer
df_clean['category_flag'] = df_clean['category_flag'].astype('string')



## Fit a basic tree model

Use just two variables, 'income' and 'house'. We'll call this the "money tree." 

What is the accuracy of the money tree? 

In [27]:
# split the datafram into independent (x) and dependent (predicted) attributes (y)
x = df_clean[['State Code']]
y = df_clean['Category']

money_tree = DecisionTreeClassifier()

# Create Decision Tree Classifer
money_tree = money_tree.fit(x,y)


## Preview the tree


In [28]:
money_tree_text = tree.export_text(money_tree)
print(money_tree_text)

|--- feature_0 <= 8.50
|   |--- feature_0 <= 3.00
|   |   |--- feature_0 <= 1.50
|   |   |   |--- class: Good
|   |   |--- feature_0 >  1.50
|   |   |   |--- class: Good
|   |--- feature_0 >  3.00
|   |   |--- feature_0 <= 5.50
|   |   |   |--- feature_0 <= 4.50
|   |   |   |   |--- class: Good
|   |   |   |--- feature_0 >  4.50
|   |   |   |   |--- class: Good
|   |   |--- feature_0 >  5.50
|   |   |   |--- feature_0 <= 7.00
|   |   |   |   |--- class: Good
|   |   |   |--- feature_0 >  7.00
|   |   |   |   |--- class: Good
|--- feature_0 >  8.50
|   |--- feature_0 <= 79.00
|   |   |--- feature_0 <= 49.50
|   |   |   |--- feature_0 <= 48.50
|   |   |   |   |--- feature_0 <= 37.50
|   |   |   |   |   |--- feature_0 <= 20.50
|   |   |   |   |   |   |--- feature_0 <= 16.50
|   |   |   |   |   |   |   |--- feature_0 <= 11.50
|   |   |   |   |   |   |   |   |--- feature_0 <= 10.50
|   |   |   |   |   |   |   |   |   |--- feature_0 <= 9.50
|   |   |   |   |   |   |   |   |   |   |--- class:

## Check Accuracy

What is the accuracy of the money_tree? Use these steps to calculate accuracy.

Is this over fitted?

In [29]:
pred = money_tree.predict(x)

#print(pred)

print("Accuracy:",metrics.accuracy_score(y, pred))

Accuracy: 0.8062750921190123


## Prune the tree

Limit the number of levels to 2

In [35]:
money_tree2 = DecisionTreeClassifier(criterion="entropy", max_depth=2)

# Create Decision Tree Classifer
money_tree2 = money_tree2.fit(x,y)


## Preview the new tree

In [36]:
money_tree2_text = tree.export_text(money_tree2)
print(money_tree2_text)

|--- feature_0 <= 8.50
|   |--- feature_0 <= 3.00
|   |   |--- class: Good
|   |--- feature_0 >  3.00
|   |   |--- class: Good
|--- feature_0 >  8.50
|   |--- feature_0 <= 79.00
|   |   |--- class: Good
|   |--- feature_0 >  79.00
|   |   |--- class: Moderate



## Re-Check Accuracy

Is this accuracy better than making a random guess?  (check the distribution above)

In [37]:
pred = money_tree2.predict(x)

print("Accuracy:",metrics.accuracy_score(y, pred))

Accuracy: 0.8062750921190123


## Visualize the model

In [38]:
from six import StringIO
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(money_tree2, out_file=dot_data, 
                   feature_names=x.columns,class_names=['State Code','Category'], 
                   filled=True,rounded=True, precision =2)

graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

IndexError: ignored